In [3]:
import scipy.stats as sc
import csv
from konlpy.tag import Twitter

twitter = Twitter()

def tokenize(sentence):
    return [''.join(m) for m in twitter.morphs(sentence)]

In [4]:
# Open dataset file
f = open('./PycharmProjects/textsimilarity/train_data.csv', 'r')
datafile = csv.reader(f)

# variable for extracted dataset from csv file
dataset = [] 

# variable for analyzed morphemes by konlpy
morphList = []

# variable for comparing words in sentence with words in morphList
tempList = []

# listing sentences 
for data in datafile:
    dataset.extend(data)

f.close()

# tokenizing and analyzing morpheme of train sentences 
tokenizedSet = [(tokenize(sentence)) for sentence in dataset]

# extending all morphemes in all sentences into one list
for tokenlist in tokenizedSet:
    tempList.append(tokenlist)
    morphList.extend(tokenlist)

In [5]:
# Open test dataset file
t = open('./PycharmProjects/textsimilarity/test_data.csv', 'r')
testdatafile = csv.reader(t)

# variable for extracted test dataset from csv file
testdataset = [] 

# variable for comparing words in sentence with words in morphList
testtempList = []

# listing sentences 
for testdata in testdatafile:
    testdataset.extend(testdata)

t.close()

# tokenizing and analyzing morpheme of test sentences 
tokenizedtestSet = [(tokenize(testsentence)) for testsentence in testdataset]

# extending all morphemes in all sentences into one list
for tokenlist in tokenizedtestSet:
    testtempList.append(tokenlist)
    morphList.extend(tokenlist)
    
#print(morphList)

# eliminate overlapped data
morphList = list(set(morphList))

#print(tempList)
#print(morphList)
print(len((morphList)))

479


In [6]:
onehotvector = [[0 for vec in morphList] for sentence in dataset]

# if sentence has word in morphList, change 0 to 1
for sentence in range(0,len(dataset)):
    
    for morph in range(0,len(morphList)):
        
        for temp in range(0,len(tempList[sentence])):
            
            if morphList[morph] == tempList[sentence][temp]:
                onehotvector[sentence][morph] = 1

print(len(onehotvector))

#print(len(testonehotvector))

300


In [7]:
testonehotvector = [[0 for vec in morphList] for sentence in testdataset]

# if sentence has word in morphList, change 0 to 1
for sentence in range(0,len(testdataset)):
    
    for morph in range(0,len(morphList)):
        
        for temp in range(0,len(testtempList[sentence])):
            
            if morphList[morph] == testtempList[sentence][temp]:
                testonehotvector[sentence][morph] = 1

print(len(testonehotvector))

99


In [8]:
def pearsoncoefficient(sentence1, sentence2):
    result = sc.stats.pearsonr(sentence1, sentence2)
    #print(result)
    return result

In [9]:
def cosinesimilarity(sentence1, sentence2) :
    vals = range(len(sentence1))
    
    sum1_sq = sum([sentence1[i]**2 for i in vals])
    sum2_sq = sum([sentence2[i]**2 for i in vals])
    
    sumofproducts = sum([sentence1[i]*sentence2[i] for i in vals])

    numerator = sumofproducts
    denominator = pow(sum1_sq, 0.5) * pow(sum2_sq, 0.5)
    
    result = numerator/denominator
    #print(result)
    return result

In [10]:

def levenshteindistance(sentence1, sentence2,key=hash):
    rows = costmatrix(sentence1, sentence2, key)
    edits = backtrace(sentence1, sentence2, rows, key)
    #print(rows[-1][-1])
    return rows[-1][-1], edits

# making cost matrix of two sentences
def costmatrix(sentence1, sentence2, key=hash):
    rows = []
    
    previous_row = range(len(sentence2) + 1)
    rows.append(list(previous_row))
    
    for i,c1 in enumerate(sentence1):
        current_row = [i+1]
        for j,c2 in enumerate(sentence2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (key(c1) != key(c2))
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
        
        rows.append(previous_row)
    #print(rows)
    return rows

def backtrace(sentence1, sentence2, rows, key=hash):
    i, j = len(sentence1), len(sentence2)
 
    edits = []
 
    while(not (i == 0  and j == 0)):
        prev_cost = rows[i][j]
 
        neighbors = []
 
        if(i!=0 and j!=0):
          neighbors.append(rows[i-1][j-1])
        if(i!=0):
          neighbors.append(rows[i-1][j])
        if(j!=0):
          neighbors.append(rows[i][j-1])
     
        min_cost = min(neighbors)
     
        if(min_cost == prev_cost):
          i, j = i-1, j-1
          edits.append({'type':'match', 'i':i, 'j':j})
        elif(i!=0 and j!=0 and min_cost == rows[i-1][j-1]):
          i, j = i-1, j-1
          edits.append({'type':'substitution', 'i':i, 'j':j})
        elif(i!=0 and min_cost == rows[i-1][j]):
          i, j = i-1, j
          edits.append({'type':'deletion', 'i':i, 'j':j})
        elif(j!=0 and min_cost == rows[i][j-1]):
          i, j = i, j-1
          edits.append({'type':'insertion', 'i':i, 'j':j})
     
    edits.reverse()
    #print(edits)
    return edits

In [11]:
def tanimoto(sentence1, sentence2):
    countIntersection = 0
    count1insentence1 = 0   
    count1insentence2 = 0
    for index in range(0,len(sentence1)):
        if sentence1[index] == 1:
            count1insentence1 = count1insentence1 + 1
        if sentence2[index] == 1:
            count1insentence2 = count1insentence2 + 1
        if (sentence1[index] == 1) and (sentence2[index] == 1):
            countIntersection = countIntersection + 1
    
    return (float)(countIntersection/(count1insentence1+count1insentence2-countIntersection))
    

In [12]:
def jaccard(sentence1, sentence2):
    countIntersection = 0
    count10case = 0
    count01case = 0
    for index in range(0,len(sentence1)):
        if sentence1[index] == 1 :
            if sentence2[index] == 1:
                countIntersection = countIntersection + 1
            elif sentence2[index] == 0:
                count10case = count10case + 1
        elif (sentence1[index] == 0) and (sentence2[index] == 1):
            count01case = count01case + 1
    return (float)(countIntersection/(count10case+count01case+countIntersection))
            

In [13]:
score_list_pearson = [[pearsoncoefficient(testsentence,sentence)[0] for sentence in onehotvector] for testsentence in testonehotvector]
print(len(score_list_pearson))
score_list_cosine = [[cosinesimilarity(testsentence,sentence) for sentence in onehotvector] for testsentence in testonehotvector]
print(len(score_list_cosine))
score_list_tanimoto = [[tanimoto(testsentence, sentence) for sentence in onehotvector] for testsentence in testonehotvector]
print(len(score_list_tanimoto))
score_list_jaccard = [[jaccard(testsentence, sentence) for sentence in onehotvector] for testsentence in testonehotvector]
print(len(score_list_jaccard))
score_list_levenshtein = [[levenshteindistance(testsentence, sentence)[0] for sentence in onehotvector] for testsentence in testonehotvector]
print(len(score_list_levenshtein))


99


99


99


99


99


In [17]:
pearson_result = []
cosine_result = []
levenshtein_result = []
tanimoto_result = []
jaccard_result = []

for index in range(0,len(testonehotvector)):
    pearson_result.append(score_list_pearson[index].index(max(score_list_pearson[index])))
    cosine_result.append(score_list_cosine[index].index(max(score_list_cosine[index])))
    levenshtein_result.append(score_list_levenshtein[index].index(min(score_list_levenshtein[index])))
    tanimoto_result.append(score_list_tanimoto[index].index(max(score_list_tanimoto[index])))
    jaccard_result.append(score_list_jaccard[index].index(max(score_list_jaccard[index])))

print(pearson_result)
print(len(pearson_result))

print(cosine_result)
print(len(cosine_result))

print(levenshtein_result)
print(len(levenshtein_result))

print(tanimoto_result)
print(len(tanimoto_result))

print(jaccard_result)
print(len(jaccard_result))

[0, 15, 8, 15, 28, 68, 68, 68, 113, 29, 30, 51, 54, 208, 49, 52, 49, 52, 49, 85, 60, 73, 84, 284, 284, 89, 33, 64, 46, 176, 90, 93, 121, 146, 119, 21, 135, 78, 115, 77, 120, 147, 179, 121, 132, 52, 146, 141, 55, 146, 176, 158, 143, 177, 151, 175, 162, 66, 176, 180, 204, 253, 193, 192, 84, 191, 206, 191, 191, 210, 233, 236, 210, 25, 10, 258, 238, 115, 221, 240, 258, 264, 264, 264, 176, 244, 258, 256, 290, 270, 285, 295, 286, 299, 258, 63, 285, 84, 285]
99
[0, 15, 8, 0, 28, 68, 68, 9, 113, 29, 30, 51, 54, 208, 49, 52, 49, 52, 49, 85, 60, 73, 84, 284, 284, 89, 33, 64, 46, 176, 90, 93, 121, 146, 119, 21, 135, 78, 115, 77, 120, 147, 164, 121, 132, 52, 146, 141, 55, 146, 176, 158, 143, 177, 179, 175, 162, 66, 176, 180, 204, 253, 193, 192, 84, 191, 206, 191, 191, 210, 233, 236, 210, 25, 10, 10, 238, 115, 221, 240, 258, 264, 264, 264, 176, 244, 258, 256, 290, 270, 285, 295, 281, 299, 258, 63, 285, 84, 285]
99
[0, 15, 8, 175, 28, 175, 175, 175, 220, 29, 30, 54, 44, 208, 49, 52, 49, 52, 49, 43, 

In [26]:
pearson_score = 0
cosine_score = 0
tanimoto_score = 0
jaccard_score = 0
levenshtein_score = 0 

for index in range(0,len(pearson_result)):
    first_index = (int)(index/10)
    if pearson_result[index] >= first_index*30 and pearson_result[index] < (first_index+1)*30:
        pearson_score = pearson_score + 1
        
print(pearson_score)

for index in range(0,len(cosine_result)):
    first_index = (int)(index/10)
    if cosine_result[index] >= first_index*30 and cosine_result[index] < (first_index+1)*30:
        cosine_score = cosine_score + 1
        
print(cosine_score)

for index in range(0,len(levenshtein_result)):
    first_index = (int) (index/10)
    if levenshtein_result[index] >= first_index*30 and levenshtein_result[index] < (first_index+1)*30:
        levenshtein_score = levenshtein_score + 1
        
print(levenshtein_score)

for index in range(0, len(tanimoto_result)):
    first_index = (int) (index/10)
    if tanimoto_result[index] >= first_index*30 and tanimoto_result[index] < (first_index+1)*30:
        tanimoto_score = tanimoto_score + 1
        
print(tanimoto_score)

for index in range(0, len(jaccard_result)):
    first_index = (int) (index/10)
    if jaccard_result[index] >= first_index*30 and jaccard_result[index] < (first_index+1)*30:
        jaccard_score = jaccard_score + 1
        
print(jaccard_score)

62
63
57
64
64
